In [40]:
!pip install -q finnhub-python

In [5]:
import requests
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()
FINNHUB_API_KEY = os.getenv("FINNHUB_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

openai_client = OpenAI(
    base_url="https://api.openai.com/v1",
    api_key=OPENAI_API_KEY
)

In [50]:
import json

function_schema = [
    {
        "name": "analyze_news_sentiment",
        "description": "Analyze the sentiment of a financial news article",
        "parameters": {
            "type": "object",
            "properties": {
                "title": {
                    "type": "string",
                    "description": "The title of the news article"
                },
                "sentiment": {
                    "type": "string",
                    "enum": ["positive", "negative", "neutral"],
                    "description": "The sentiment of the article"
                },
                "score": {
                    "type": "number",
                    "minimum": 0,
                    "maximum": 100,
                    "description": "Sentiment score between 0 and 100 with one decimal places. 0.0 being most negative and 100.0 being most positive"
                },
                "related_score": {
                    "type": "number",
                    "minimum": 0,
                    "maximum": 10,
                    "description": "How related the article to the company. 0 being not related at all and 10 being very related"
                }
            },
            "required": ["title", "sentiment", "score", "related_score"]
        }
    }
]


def invoke_openai(prompt, model = 'gpt-4o-mini'):
    for i in range(3):
        try:
            kwargs = {
                    "model": model,
                    "messages": [{"role": "user", "content": prompt}],
                    "temperature": 0.0,
                    # "response_format": {"type": "json_object"},
                    "functions": function_schema,  # Add the function schema
                    "function_call": {"name": "analyze_news_sentiment"}  # Force the function call        
            }



            response = openai_client.chat.completions.create(**kwargs)
            function_response = response.choices[0].message.function_call.arguments
            return json.loads(function_response)
        
        except Exception as e:
            print(f"Error: {e}")
            if i == 2:
                print("FAILEDDDDD \n\n\n\n")
                return None


In [56]:
news_title = "Stock-Split Watch: 1 Under-the-Radar Growth Stock Up 510% Since the Beginning of 2023, Spotify has made its investors a fortune in the past two years. "
result = invoke_openai(f"Analyze the sentiment of this article for the company Google(GOOG): {news_title}")


if result:
    print(f"Title: {result['title']}")
    print(f"Sentiment: {result['sentiment']}")
    print(f"Score: {result['score']}")
    print(f"Related Score: {result['related_score']}")

Title: Stock-Split Watch: 1 Under-the-Radar Growth Stock Up 510% Since the Beginning of 2023, Spotify has made its investors a fortune in the past two years.
Sentiment: positive
Score: 75.0
Related Score: 2


In [42]:
import finnhub

finnhub_client = finnhub.Client(api_key=FINNHUB_API_KEY)

response = finnhub_client.company_news('GOOG', _from="2024-11-28", to="2024-11-29")


In [66]:
response[10]

{'category': 'company',
 'datetime': 1732788000,
 'headline': 'Stock-Split Watch: 1 Under-the-Radar Growth Stock Up 510% Since the Beginning of 2023',
 'id': 131682728,
 'image': 'https://g.foolcdn.com/editorial/images/798924/billionaire.jpg',
 'related': 'GOOG',
 'source': 'Yahoo',
 'summary': 'This leading streaming company has made its investors a fortune in the past two years.',
 'url': 'https://finnhub.io/api/news?id=b456a67d2ad55cd0839b57bf0f4fc268acd6b98e1fada1649d90f92182819599'}

In [68]:
!pip install -q -U newspaper4k

In [60]:
def scrape_site(url: str) -> str:
    response = requests.get("https://r.jina.ai/" + url)
    return response.text

with open("temp.txt", "w") as f:
    f.write(scrape_site(response[5]['url']))



In [81]:
temp = scrape_site(response[10]['url'])
invoke_openai(f"Analyze the sentiment of this article for the company Google(GOOG): {temp}")

{'title': 'Stock-Split Watch: 1 Under-the-Radar Growth Stock Up 510% Since the Beginning of 2023',
 'sentiment': 'positive',
 'score': 75.0,
 'related_score': 8}

In [79]:
from newspaper import Article

def newspaper_api(url: str):
    article = Article(url)
    article.download()
    article.parse()
    

    return article.text

In [80]:
newspaper_api(response[10]['url'])

'Stock splits have been in vogue recently. Large technology companies like Amazon, Alphabet, Nvidia, and Tesla have split their stocks after seeing their share prices get close to $1,000 or more. This financial engineering tactic didn\'t change anything about these stocks\' underlying businesses, but it can make it easier for individual investors to buy a single share.\n\nOne potential stock-split candidate is Spotify (SPOT -0.29%). The audio streaming leader is up over 500% since the start of 2023, and the stock is now approaching $500 per share, or stock-split territory. Here\'s why the stock has made a massive turnaround, and whether you should buy shares of Spotify after its big gains.\n\nEfficient spending, expanding margins\n\nMany investors will know Spotify as it is one of the most popular music and podcast streaming services worldwide. With 640 million monthly active users (MAUs), perhaps only YouTube has a larger reach around the world in this niche.\n\nSpotify mostly makes m

In [91]:
from bs4 import BeautifulSoup
def extract_article_text(url):
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')

    # Try different methods to extract article content based on structure
    article = soup.find('div', class_='article-body')  # Adjust based on actual website structure

    if not article:
        # Try using another common structure
        article = soup.find('section', {'class': 'content'})  # Adjust for another tag/class
    if not article:
        # Try using CSS selectors to grab all text in relevant elements
        article = soup.select('div.article-body p, div.article-body h1')

    if article:
        article_text = " ".join([p.get_text(strip=True) for p in article])
        return article_text
    else:
        print("No article text found.")
        return None

In [95]:
for i in range(10):
    print(response[i]['url'])
    print(f"Article {i+1}: {extract_article_text(response[i]['url'])}")

https://finnhub.io/api/news?id=41b103581ea2424564a0a2e2187afe9a5e8b6181f9ad5b0b08fe8e712b5f6105
No article text found.
Article 1: None
https://finnhub.io/api/news?id=c93c3ddae129c3bade6df7631f64a4266cc1e1b5caa7238538fbe8b4891dc3dd
Failed to retrieve the webpage. Status code: 403
Article 2: None
https://finnhub.io/api/news?id=b078074bac0dd975ab786b8568293484d566b101708e756f77639b94e1a275b1
Failed to retrieve the webpage. Status code: 403
Article 3: None
https://finnhub.io/api/news?id=ec9f5a9e406757a7f960793a80e0201c51a0e924024543922a3b0c3789976ab6
Failed to retrieve the webpage. Status code: 403
Article 4: None
https://finnhub.io/api/news?id=28fa8f486cec22f632895bc846bce32d1ab54a871a5ceef0e95383df0ad71449
No article text found.
Article 5: None
https://finnhub.io/api/news?id=2cb58bcb156dc3d11a35cf70b1a756a2cc507dff95cc54ec63bde94da5a67922
No article text found.
Article 6: None
https://finnhub.io/api/news?id=222559de650ff69a8b165e82829a2791208d66834b9c328fdfaae71419f4e915
AAA
Article 7:  